In [3]:
from pyspark.sql import SparkSession

In [4]:
# OutOfMemory 오류가 나면 각종 설정을 추가적으로 해줄 수 있다.
MAX_MEMORY = '5g'
spark = SparkSession.builder.appName("movie-recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

In [5]:
directory="C:/Users/wjswl/study_spark/data"
filename = "ratings.csv"

In [7]:
ratings_df = spark.read.csv(f"file:///{directory}\\{filename}", inferSchema=True, header=True)
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



timestamp만 빼고 선택해 주기

In [8]:
ratings_df = ratings_df.select(["userid", "movieId", "rating"])
ratings_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [9]:
ratings_df.select('rating').describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423508|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



`train`, `test` 데이터 세트 분리하기

In [10]:
train_ratio = 0.8
test_ratio  = 0.2

train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

## ALS 추천 알고리즘 가져오기

In [11]:
from pyspark.ml.recommendation import ALS

In [12]:
als = ALS(
    maxIter=5,
    regParam=0.1,
    userCol = "userid",
    itemCol = "movieId",
    ratingCol = "rating",
    coldStartStrategy="drop"
)

In [13]:
model = als.fit(train_df)

**예측**

In [14]:
predictions = model.transform(test_df)
predictions.show()

+------+-------+------+----------+
|userid|movieId|rating|prediction|
+------+-------+------+----------+
|    31|   1580|   3.0| 2.2900095|
|    31|   6620|   1.5| 2.5070682|
|    76|   1959|   5.0| 3.5795574|
|   159|  54190|   5.0| 4.0784464|
|   296|   1580|   3.0| 2.7322464|
|   321|   1580|   3.0| 3.0947466|
|   497|   1580|   5.0| 3.2466366|
|   513|  44022|   5.0| 4.3242855|
|   597|   1645|   5.0| 3.4671247|
|   606|  36525|   2.5| 4.2295327|
|   626|   2122|   2.0| 2.2905457|
|   626|   6658|   3.5| 2.8782196|
|   626|  36525|   4.0| 3.3931115|
|   772|    471|   4.0|  3.410218|
|   772|   1645|   3.0| 2.9847307|
|   830|   1580|   5.0| 3.5145903|
|   844|   1342|   4.0| 2.9084284|
|   847|   1088|   1.0| 2.5815601|
|   847|  96488|   4.0| 3.7842422|
|   847| 119432|   3.5|  3.204154|
+------+-------+------+----------+
only showing top 20 rows



* `rating` : 실제 값( `target`, `label` )
* `prediction` : 모델이 예측한 값

**통계 확인**

In [15]:
predictions.select("rating", "prediction").describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4997668|           4997668|
|   mean|  3.53423116541555|3.4271337235375023|
| stddev|1.0607227539631854|0.6455315965581465|
|    min|               0.5|        -1.4841435|
|    max|               5.0|          6.953022|
+-------+------------------+------------------+



**RMSE Evaluation**

$$
MSE = \frac{1}{N}\sum_{i=1}^{N}(y_i - t_i)^2
$$

MSE : 평균 제곱 오차 (Mean Squared Error)
- $y_i$ : 예측 값 ($\hat{y}$)
- $t_i$ : 실제 값

$$
RMSE = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_i - t_i)^2}
$$

RMSE : 평균 제곱 오차의 제곱근 (Root Mean Squared Error)

In [16]:
# 영화의 평점 예측, 즉 회귀를 진행 했기 때문에 RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [17]:
rmse = evaluator.evaluate(predictions)
print(rmse)

0.8098989350379645


평균적으로 예측을 했을 때 `0.82`점 정도의 오차가 있다.

`user` 또는 `movie`마다 추천을 해주기

In [18]:
# 각 user 에게 top3 아이템을 추천 = {item 번호, 예측된 점수}
model.recommendForAllUsers(3).show()

C:\Users\wjswl\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userid|     recommendations|
+------+--------------------+
|    26|[{177209, 5.58156...|
|    27|[{177411, 6.41921...|
|    28|[{177209, 7.57901...|
|    31|[{194334, 3.74085...|
|    34|[{177209, 5.64509...|
|    44|[{177209, 7.05599...|
|    53|[{194334, 6.61187...|
|    65|[{177209, 6.58599...|
|    76|[{177209, 5.92992...|
|    78|[{177209, 7.23135...|
|    81|[{153184, 5.00843...|
|    85|[{181803, 5.74279...|
|   101|[{194334, 5.01973...|
|   103|[{177209, 6.02106...|
|   108|[{177209, 5.29794...|
|   115|[{177209, 6.03530...|
|   126|[{177209, 6.29277...|
|   133|[{192089, 5.44591...|
|   137|[{205453, 5.44152...|
|   148|[{194434, 5.66685...|
+------+--------------------+
only showing top 20 rows



In [19]:
# 각 movie에 어울리는 top 3 user를 추천
model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     12|[{87426, 5.497687...|
|     26|[{105801, 5.01915...|
|     27|[{87426, 5.831301...|
|     28|[{103920, 5.54338...|
|     31|[{87426, 5.630018...|
|     34|[{58248, 5.256312...|
|     44|[{87426, 5.292686...|
|     53|[{84112, 5.582429...|
|     65|[{87426, 5.409262...|
|     76|[{87426, 5.377369...|
|     78|[{67467, 4.667077...|
|     81|[{161312, 4.73326...|
|     85|[{80084, 4.931544...|
|    101|[{89560, 4.930403...|
|    103|[{87426, 5.247668...|
|    108|[{29663, 5.618244...|
|    115|[{87426, 5.975257...|
|    126|[{87426, 5.069078...|
|    133|[{31506, 5.429294...|
|    137|[{31506, 4.997553...|
+-------+--------------------+
only showing top 20 rows



**user_list**를 이용해서 예측

In [20]:
from pyspark.sql.types import IntegerType

user_list = [65, 78, 81]
users_df = spark.createDataFrame(user_list, IntegerType()).toDF("userId")

users_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [21]:
# 데이터 프레임으로 예측 할 때는 recommendForUserSubset
user_recs = model.recommendForUserSubset(users_df, 5) # 각 user에 대해 top 5 추천
user_recs.show()

C:\Users\wjswl\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userid|     recommendations|
+------+--------------------+
|    65|[{177209, 6.58599...|
|    78|[{177209, 7.23135...|
|    81|[{153184, 5.00843...|
+------+--------------------+



`65`번 사용자를 위한 추천 영화 목록 만들어 보기

In [22]:
movies_list = user_recs.collect()[0].recommendations
movies_list

[Row(movieId=177209, rating=6.585999488830566),
 Row(movieId=139036, rating=6.525178909301758),
 Row(movieId=151989, rating=6.065602779388428),
 Row(movieId=194434, rating=6.048390865325928),
 Row(movieId=192261, rating=5.912269115447998)]

In [23]:
recs_df = spark.createDataFrame(movies_list)
recs_df.show()

+-------+-----------------+
|movieId|           rating|
+-------+-----------------+
| 177209|6.585999488830566|
| 139036|6.525178909301758|
| 151989|6.065602779388428|
| 194434|6.048390865325928|
| 192261|5.912269115447998|
+-------+-----------------+



영화 이름으로 추천하기

In [24]:
movies_file = "movies.csv"

In [26]:
movies_df = spark.read.csv(f"file:///{directory}\\{movies_file}", inferSchema=True, header=True)
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

spark SQL을 사용하기 위해 TempView 등록

In [30]:
recs_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView("movies")

추천 영화와 추천 영화의 제목, 장르 조회

In [31]:
query = """
    
    SELECT * 
    
    FROM movies
    JOIN recommendations ON movies.movieId = recommendations.movieId
    
    ORDER BY rating desc
    
"""

recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+------------------+-------+-----------------+
|movieId|               title|            genres|movieId|           rating|
+-------+--------------------+------------------+-------+-----------------+
| 177209|      Acı Aşk (2009)|             Drama| 177209|6.585999488830566|
| 139036|World Gone Wild (...|     Action|Sci-Fi| 139036|6.525178909301758|
| 151989|    The Thorn (1971)|            Comedy| 151989|6.065602779388428|
| 194434|   Adrenaline (1990)|(no genres listed)| 194434|6.048390865325928|
| 192261|Don't Laugh at My...|      Comedy|Drama| 192261|5.912269115447998|
+-------+--------------------+------------------+-------+-----------------+



In [32]:
# 함수화 시켜보기
def get_recommendations(user_id, num_recs):
    user_df = spark.createDataFrame([user_id], IntegerType()).toDF("userId")
    user_recs_df = model.recommendForUserSubset(user_df, num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list)
    
    recommended_movies = recs_df.join(movies_df, "movieId")
    return recommended_movies

In [33]:
recs = get_recommendations(456, 10)
recs.toPandas()

,movieId,rating,title,genres
0,177209,6.539916,Acı Aşk (2009),Drama
1,199187,6.343500,Hoaxed (2019),(no genres listed)
2,139036,6.314621,World Gone Wild (1988),Action|Sci-Fi
3,180323,6.157449,The Magnificent Scoundrels (1991),Comedy
4,151615,6.084562,Hello Stranger (2010),Drama
5,173871,6.081365,I'll Take You There (1999),Comedy|Drama|Romance
6,167932,6.077082,Eye of the Eagle (1987),Action|Thriller
7,145871,6.077082,The Grasshopper (1967),Comedy|Romance
8,204302,5.991782,Ek Ladki Ko Dekha Toh Aisa Laga (2019),Comedy|Drama|Romance
9,175635,5.985383,American Teacher (2011),Documentary


In [34]:
spark.stop()